# Problem 2

## Data Import
First, download the movielens (small) [dataset](https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) as `pandas.DataFrame` objects. 

In [1]:
import pandas as pd

path = "Misc_files/movielens_data/ml-latest-small/"

# load movies and ratings DataFrames
movies = pd.read_csv(path+"movies.csv", header=0)
ratings = pd.read_csv(path+"ratings.csv", header=0)

We can then use the `head()` method to see the raw format of these `DataFrame` objects.

In [2]:
n_movies = len(movies)

print(f"Number of Unique Movies: {n_movies}")
movies.head()

Number of Unique Movies: 9742


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
n_ratings = len(ratings)
n_users = ratings.userId.nunique()
n_rated_movies = ratings.movieId.nunique()

print(f"Number of Ratings: {n_ratings}\nNumber of Users: {n_users}\nNumber of Unique Rated Movies: {n_rated_movies}")
ratings.head()

Number of Ratings: 100836
Number of Users: 610
Number of Unique Rated Movies: 9724


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Upon inspection of the raw data we note that of the 9,742 movies in the `movies` DataFrame, only 9,724 movies have been rated.

## Preprocessing

### Embedding Matrix $X$
To obtain the concurrent number of likes $X_{i,j}$ we must first binary encode (`0` or `1`) each `"rating"` in the `ratings` DataFrame. Let us encode the value of liking a movie for each review as such

$$ \text{Liked}(\text{Rating}) =
    \begin{cases}
        1 & \text{if Rating}\geq 4\\
        0 & \text{otherwise}
    \end{cases}$$

and store these values in a new `"liked"` column. We can subsequently drop the unnecessary `rating` and `timestamp` columns after this process.

In [4]:
import numpy as np

# create liked column
ratings["liked"] = np.where(ratings["rating"] >= 4, 1, 0)

# drop columns
ratings.drop(["rating", "timestamp"], axis=1, inplace=True)

We next create the `movie_ratings` DataFrame by joing the `movies` and `ratings` DataFrames. Setting the `merge` method parameter `how="left"` ensures that the original number of movies, 9,742, are maintained after the join.

In [5]:
# left join on movieId
movie_ratings = pd.merge(movies, ratings, how="left", on="movieId").reset_index()

A user-likes interaction matrix can then be constructed using the `pivot_table` method, whose rows correspond to the number of unique users `n_users` and columns correspond to the number of unique movies `n_movies` from the original data. This results in a sparse matrix whose rows summarize each users liked movies.

In [6]:
# pivot table on userId
user_likes = movie_ratings.pivot_table(values="liked", index="userId", columns="movieId", dropna=False, fill_value=0)

user_likes

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1.0,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
607.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


Embedding matrix $X$ can now be constructed as the inner (dot) product of the transpose of `user_likes ` and itself. Element $X_{ij} \in X$ corresponds to the number of users that like both movie $i$ and $j$.

In [7]:
# convert to numpy ndarray for dot product computation
user_likes_array = user_likes.to_numpy()

# create X
X_embedding = np.dot(user_likes_array.T, user_likes_array)

# fill diagonals of X with zeros
np.fill_diagonal(X_embedding, 0)

# display as DataFrame for clarity
X_display = pd.DataFrame(X_embedding, index=movies.movieId, columns = movies.movieId)

X_display

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,21,11,0,7,27,7,1,4,19,...,0,0,0,0,0,0,0,0,0,0
2,21,0,5,0,4,8,6,0,0,9,...,0,0,0,0,0,0,0,0,0,0
3,11,5,0,0,4,4,5,1,2,3,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,7,4,4,0,0,3,4,1,1,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
193583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
193585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model Architecture

With preprocessing completed, we convert embedding $X$ into a `torch` tensor object and construct cost function $c$.

In [8]:
import torch
from torch import mps

# make torch deterministic for reproducibility
torch.manual_seed(576)

# set device
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

X = torch.tensor(X_embedding, dtype = torch.float32)

We implement the below cost function:
$$c(v_1,\ldots, v_M)=\sum_{i=1}^M\sum_{j=1}^M 1_{[i\neq j]}(v_i^T v_j-X_{i,j})^2$$
by creating a superclass of the `torch` `nn.Module` class.

In [9]:
import torch.nn as nn

class cost_function(nn.Module):
    def __init__(self):
        super().__init__()
    
    @ classmethod
    def forward(self, v, X):
        # perform main cost function
        costs = ((1 - torch.eye(n_movies)) * (torch.mm(v.t(), v) - X)**2).sum()
        
        return costs

In [10]:
import torch.optim as optim

# SGD sees high numerical instability w.r.t. parameter initialization. We tighten the range of initial parameters thusly.
v = torch.normal(mean=0, std=0.5, size=(n_movies,n_movies), requires_grad=True)

# training loop
model = cost_function()
model.to(device)

optimizer = optim.Adam([v], 0.5)


In [11]:
from tqdm.notebook import tqdm

def train(model, v, X, optimizer, n_epochs=100, verbose = 10):
    for epoch in tqdm(range(n_epochs)):
        
        costs = []
        
        # forward pass
        cost = model.forward(v, X)
        
        # backward pass
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        costs.append(cost)
        
        if epoch % verbose == 0:
            print(f"Epoch: {epoch}\tCost: {cost: .5f}")
            
    return costs

In [12]:
history = train(model, v, X, optimizer, 100)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0	Cost:  57846087680.00000
Epoch: 10	Cost:  8292660224.00000
Epoch: 20	Cost:  2230697216.00000
Epoch: 30	Cost:  607125888.00000
Epoch: 40	Cost:  173342816.00000
Epoch: 50	Cost:  51432160.00000
Epoch: 60	Cost:  16213693.00000
Epoch: 70	Cost:  5517181.50000
Epoch: 80	Cost:  2026323.50000
Epoch: 90	Cost:  787639.37500


In [13]:
import torch.optim as optim

# SGD sees high numerical instability w.r.t. parameter initialization. We tighten the range of initial parameters thusly.
v = torch.normal(mean=0, std=0.5, size=(n_movies,n_movies), requires_grad=True)

# training loop
model = cost_function()
model.to(device)

optimizer = optim.SGD([v], 1e-5, momentum=0.9)


1. Need to evaluate model on multiple hyper parameters:

different learning rates
different optimizers – Adam and SGD

We need two different plots for costs/epoch iterations, one for Adam and one for SGD.

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# List of hyperparameters to try
learning_rates = [0.001, 0.01, 0.1]  # Different learning rates
optimizers = ['Adam', 'SGD']  # Different optimizers

# Store results and histories separately for each optimizer
results = []
histories = []

# Loop over hyperparameters
for lr in learning_rates:
    for optimizer_name in optimizers:
        # Create a new model for each configuration
        v = torch.normal(mean=0, std=0.5, size=(n_movies, n_movies), requires_grad=True)
        model = cost_function()
        model.to(device)

        # Define the optimizer based on the chosen name
        if optimizer_name == 'Adam':
            optimizer = optim.Adam([v], lr=lr)
        elif optimizer_name == 'SGD':
            optimizer = optim.SGD([v], lr=lr, momentum=0.9)

        # Train the model and store the history
        history = train(model, v, X, optimizer, n_epochs=100)
        histories.append((lr, optimizer_name, history))

        # Evaluate the model (you can use a validation set or any other evaluation metric)
        # For demonstration purposes, we'll just store the final cost here
        final_cost = history[-1]

        # Store the results
        result = {
            'learning_rate': lr,
            'optimizer': optimizer_name,
            'final_cost': final_cost.item()
        }
        results.append(result)

# Plot the cost vs. epoch iterations for each optimizer
for lr, optimizer_name, history in histories:
    plt.plot(range(len(history)), history, label=f"{optimizer_name} (LR={lr})")

plt.xlabel('Epochs')
plt.ylabel('Cost')
plt.legend()
plt.title('Cost vs. Epoch Iterations')
plt.show()


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0	Cost:  57829216256.00000
Epoch: 10	Cost:  52843864064.00000
Epoch: 20	Cost:  48331300864.00000
Epoch: 30	Cost:  44281692160.00000
Epoch: 40	Cost:  40660541440.00000
Epoch: 50	Cost:  37422321664.00000
Epoch: 60	Cost:  34520301568.00000
Epoch: 70	Cost:  31911553024.00000
Epoch: 80	Cost:  29558611968.00000
Epoch: 90	Cost:  27429482496.00000


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0	Cost:  57861603328.00000
Epoch: 10	Cost:  nan
Epoch: 20	Cost:  nan
Epoch: 30	Cost:  nan
Epoch: 40	Cost:  nan
Epoch: 50	Cost:  nan


KeyboardInterrupt: 

In [ ]:
'''
Important!!

this implementation using the custom cost_function() class makes changes to the input vector v.
If you want to store v, say in a list or as a new variable for later, use the torch method .clone() to 
create a copy for later use.
'''

'\nImportant!!\n\nthis implementation using the custom cost_function() class makes changes to the input vector v.\nIf you want to store v, say in a list or as a new variable for later, use the torch method .clone() to \ncreate a copy for later use.\n'

Need to complete Problem 2, recommendation system and comparison between optimizers.

In [ ]:
# # After training the model and obtaining the embeddings (X_embedding)

# # Calculate movie similarities using cosine similarity
# from sklearn.metrics.pairwise import cosine_similarity

# # Calculate cosine similarity between movie embeddings
# movie_similarities = cosine_similarity(X_embedding)

# # Function to recommend top N movies based on input movies
# def recommend_movies(input_movies, movie_names, movie_similarities, top_n=10):
#     # Find the indices of input movies
#     input_indices = [movie_names.index(movie) for movie in input_movies]

#     # Calculate the average similarity scores for input movies
#     input_similarity_scores = np.mean(movie_similarities[input_indices], axis=0)

#     # Exclude input movies from consideration
#     for index in input_indices:
#         input_similarity_scores[index] = -1.0  # Set similarity score to a negative value

#     # Get indices of top N recommended movies
#     top_indices = np.argsort(input_similarity_scores)[-top_n:][::-1]

#     # Get the names of recommended movies
#     recommended_movies = [movie_names[i] for i in top_indices]

#     return recommended_movies

In [ ]:
# # Define the input movies
# input_movies = ["Apollo 13 (1995)", "Toy Story (1995)", "Home Alone (1990)"]

# # Get recommended movies
# recommended_movies = recommend_movies(input_movies, movies['title'].tolist(), movie_similarities)

# # Print the recommended movies
# print("Recommended Movies:")
# for i, movie in enumerate(recommended_movies, start=1):
#     print(f"{i}. {movie}")


Recommended Movies:
1. Toy Story (1995)
2. Aladdin (1992)
3. Jurassic Park (1993)
4. Apollo 13 (1995)
5. Lion King, The (1994)
6. Forrest Gump (1994)
7. Braveheart (1995)
8. Terminator 2: Judgment Day (1991)
9. Fugitive, The (1993)
10. Independence Day (a.k.a. ID4) (1996)


Changing the learning rate or optimizer can affect how the model updates its parameters during training, leading to different embeddings in the embedding space. As a result, the recommendations based on these embeddings can vary. To find the optimal hyperparameters for your recommendation model, it's common to perform hyperparameter tuning and experiment with different learning rates and optimizers to see which combination yields the best recommendations